### 0. 라이브러리 호출 & 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import os
import math
from glob import glob
import pandas as pd
import cv2

### 1. 훈련/테스트 데이터 로드 함수

In [28]:
def get_train_data(data_dir):
    input_path_list = []
    out_path_list = []
    # 경로를 담을 빈 리스트 생성
    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):
            if case_name == 'input':
                # get input path
                input_path_list.extend(glob(os.path.join(current_path, '*.jpg')))
                input_path_list.extend(glob(os.path.join(current_path, '*.png')))
            elif case_name == 'output':
                # get output path
                out_path_list.extend(glob(os.path.join(current_path, '*.jpg')))
                out_path_list.extend(glob(os.path.join(current_path, '*.png')))
    return input_path_list, out_path_list
    # 데이터를 분리해서 반환

In [29]:
train_input_path, train_output_path = get_train_data('./mini_crack_segmentation_dataset/train')